**IF-B || Kecerdasan Buatan**<br>
Nama Kelompok: <br>
Jonathan Natannael Zefanya <br>
Daffa Danindra <br>
Ikhwan El-Faris <br>
Dhetalisa Aulia Kirana <br>

In [1]:
from google.colab import files

# Upload data training
print("Upload Data Training")
uploaded_train = files.upload()

# Upload data testing
print("Upload Data Testing")
uploaded_test = files.upload()

Upload Data Training


Saving Data_Training_AdultIncome.csv to Data_Training_AdultIncome.csv
Upload Data Testing


Saving Data_Testing_AdultIncome.csv to Data_Testing_AdultIncome.csv


In [2]:
# Fungsi untuk membuat tabel saat membaca data
from IPython.display import display, HTML

def tabel(df):
    display(HTML(df.to_html(index=False)))

In [3]:
# Membaca dan Menampilkan data training dan data testing
import pandas as pd

#Data Training
train_data = pd.read_csv(next(iter(uploaded_train.keys())))
print("Data Training:")
tabel(train_data)

#Data Testing
test_data = pd.read_csv(next(iter(uploaded_test.keys())))
print("Data Testing:")
tabel(test_data)

Data Training:


age,workclass,education,marital-status,occupation,relationship,gender,hours-per-week,native-country,income
30 - 39,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,Male,40,United-States,<=50K
20 - 29,Private,11th,Married-civ-spouse,Transport-moving,Husband,Male,40,United-States,<=50K
40 - 49,Private,HS-grad,Married-civ-spouse,Adm-clerical,Husband,Male,40,United-States,<=50K
30 - 39,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Male,40,United-States,>50K
60 - 69,Private,Some-college,Married-civ-spouse,Handlers-cleaners,Husband,Male,15,United-States,<=50K
50 - 59,Private,HS-grad,Married-civ-spouse,Handlers-cleaners,Husband,Male,40,United-States,>50K
40 - 49,Local-gov,Doctorate,Never-married,Prof-specialty,Not-in-family,Female,40,United-States,>50K
30 - 39,Private,Some-college,Never-married,Craft-repair,Not-in-family,Male,40,United-States,<=50K
30 - 39,State-gov,HS-grad,Married-civ-spouse,Protective-serv,Other-relative,Female,40,United-States,<=50K
50 - 59,Private,10th,Divorced,Other-service,Not-in-family,Female,20,United-States,<=50K


Data Testing:


age,workclass,education,marital-status,occupation,relationship,gender,hours-per-week,native-country,income
20 - 29,Private,Some-college,Never-married,Other-service,Own-child,Male,15,United-States,<=50K
20 - 29,Local-gov,Some-college,Never-married,Protective-serv,Own-child,Female,40,United-States,<=50K
50 - 59,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Male,40,United-States,<=50K
30 - 39,Private,HS-grad,Divorced,Adm-clerical,Unmarried,Female,40,United-States,<=50K
40 - 49,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Male,40,United-States,>50K
30 - 39,Local-gov,Some-college,Married-civ-spouse,Craft-repair,Husband,Male,40,United-States,<=50K
20 - 29,Private,Bachelors,Never-married,Prof-specialty,Own-child,Female,72,United-States,<=50K
20 - 29,Private,HS-grad,Never-married,Sales,Not-in-family,Female,35,United-States,<=50K
40 - 49,Private,Some-college,Widowed,Adm-clerical,Not-in-family,Female,60,United-States,<=50K
40 - 49,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,Male,45,United-States,<=50K


In [4]:
# Menghitung probabilitas dari setiap target
# P(Ci)
probabilitas = train_data['income'].value_counts(normalize=True)
print(f"Probabilitas Prior:\n{probabilitas}")

Probabilitas Prior:
income
<=50K    0.764706
>50K     0.235294
Name: proportion, dtype: float64


In [5]:
# Menghitung probabilitas kondisional (P(X|Ci)) dari setiap atribut yang diberikan target tertentu
probabilitas_kondisional = {}
atributs = train_data.columns[:-1]  # Semua kolom kecuali kolom 'income'

In [6]:
for atribut in atributs:
    probabilitas_kondisional[atribut] = {}
    # Menghitung probabilitas kondisional untuk setiap nilai pada atribut
    for value in train_data[atribut].unique():
      # Menghitung P(X|Ci) = Total atribut sesuai dengan jenis "income"/total jenis "income"
        probabilitas_kondisional[atribut][value] = train_data.groupby('income')[atribut].apply(lambda x: (x == value).sum() / x.count()).to_dict()

In [7]:
print("Probabilitas Kondisional:\n", probabilitas_kondisional)

Probabilitas Kondisional:
 {'age': {'30 - 39': {'<=50K': 0.23076923076923078, '>50K': 0.16666666666666666}, '20 - 29': {'<=50K': 0.358974358974359, '>50K': 0.25}, '40 - 49': {'<=50K': 0.20512820512820512, '>50K': 0.3333333333333333}, '60 - 69': {'<=50K': 0.10256410256410256, '>50K': 0.08333333333333333}, '50 - 59': {'<=50K': 0.07692307692307693, '>50K': 0.16666666666666666}, '70 - 79': {'<=50K': 0.02564102564102564, '>50K': 0.0}}, 'workclass': {'Private': {'<=50K': 0.6923076923076923, '>50K': 0.5833333333333334}, 'Local-gov': {'<=50K': 0.10256410256410256, '>50K': 0.25}, 'State-gov': {'<=50K': 0.1282051282051282, '>50K': 0.08333333333333333}, 'Self-emp-not-inc': {'<=50K': 0.05128205128205128, '>50K': 0.08333333333333333}, '?': {'<=50K': 0.02564102564102564, '>50K': 0.0}}, 'education': {'Some-college': {'<=50K': 0.28205128205128205, '>50K': 0.08333333333333333}, '11th': {'<=50K': 0.02564102564102564, '>50K': 0.0}, 'HS-grad': {'<=50K': 0.3076923076923077, '>50K': 0.25}, 'Bachelors': {'<=

In [15]:
# Fungsi klasifikasi menggunakan Naive Bayes
def naive_bayes(instance):
    target = probabilitas.index
    probabilities = {}

    for tgt in target:
        prob = probabilitas[tgt] # P(Ci)
        for atribut in atributs:
            if instance[atribut] in probabilitas_kondisional[atribut]: # Probabilitas Kondisional (P(X|Ci))
                # P(Ci|X) = P(Ci) * P(X|Ci)
                prob *= probabilitas_kondisional[atribut][instance[atribut]].get(tgt, 1e-6)  # Tambahkan smoothing dengan nilai sangat kecil
            else:
                prob *= 1e-6  # Tambahkan smoothing dengan nilai sangat kecil
        probabilities[tgt] = prob

    predicted_target = max(probabilities, key=probabilities.get)
    return predicted_target, probabilities

In [16]:
# Klasifikasi data pengujian
results = test_data.apply(naive_bayes, axis=1)

In [17]:
# Membuat hasil ke dalam dataframe
test_data['predicted_target'] = results.apply(lambda x: x[0])
test_data['probabilities'] = results.apply(lambda x: x[1])

In [18]:
# Cetak semua informasi probabilitas dari semua jenis target
for index, row in test_data.iterrows():
    print(f"Data ke-{index + 1} dengan atribut {row[atributs].to_dict()}")
    for tgt, prob in row['probabilities'].items():
        print(f"  Probabilitas dari target {tgt}: {prob}")
    print(f"  Prediksi Target: {row['predicted_target']}\n")

Data ke-1 dengan atribut {'age': '20 - 29', 'workclass': 'Private', 'education': 'Some-college', 'marital-status': 'Never-married', 'occupation': 'Other-service', 'relationship': 'Own-child', 'gender': 'Male', 'hours-per-week': 15, 'native-country': 'United-States'}
  Probabilitas dari target <=50K: 1.9194110814056596e-05
  Probabilitas dari target >50K: 0.0
  Prediksi Target: <=50K

Data ke-2 dengan atribut {'age': '20 - 29', 'workclass': 'Local-gov', 'education': 'Some-college', 'marital-status': 'Never-married', 'occupation': 'Protective-serv', 'relationship': 'Own-child', 'gender': 'Female', 'hours-per-week': 40, 'native-country': 'United-States'}
  Probabilitas dari target <=50K: 3.007867241972325e-06
  Probabilitas dari target >50K: 0.0
  Prediksi Target: <=50K

Data ke-3 dengan atribut {'age': '50 - 59', 'workclass': 'Private', 'education': 'HS-grad', 'marital-status': 'Married-civ-spouse', 'occupation': 'Machine-op-inspct', 'relationship': 'Husband', 'gender': 'Male', 'hours-pe

In [19]:
# Menampilkan hasil
print("Hasil Klasifikasi:")
tabel(test_data[["age","workclass","education","marital-status","occupation","relationship","gender","hours-per-week","native-country", 'predicted_target', 'probabilities']])

Hasil Klasifikasi:


age,workclass,education,marital-status,occupation,relationship,gender,hours-per-week,native-country,predicted_target,probabilities
20 - 29,Private,Some-college,Never-married,Other-service,Own-child,Male,15,United-States,<=50K,"{'<=50K': 1.9194110814056596e-05, '>50K': 0.0}"
20 - 29,Local-gov,Some-college,Never-married,Protective-serv,Own-child,Female,40,United-States,<=50K,"{'<=50K': 3.007867241972325e-06, '>50K': 0.0}"
50 - 59,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Male,40,United-States,<=50K,"{'<=50K': 2.4790315850102967e-05, '>50K': 0.0}"
30 - 39,Private,HS-grad,Divorced,Adm-clerical,Unmarried,Female,40,United-States,<=50K,"{'<=50K': 8.899087741062604e-07, '>50K': 0.0}"
40 - 49,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,Male,40,United-States,>50K,"{'<=50K': 4.131719308350494e-05, '>50K': 0.0021413442295213697}"
30 - 39,Local-gov,Some-college,Married-civ-spouse,Craft-repair,Husband,Male,40,United-States,>50K,"{'<=50K': 1.5149637463951818e-05, '>50K': 0.00012236252740122108}"
20 - 29,Private,Bachelors,Never-married,Prof-specialty,Own-child,Female,72,United-States,<=50K,"{'<=50K': 3.1757528801439085e-11, '>50K': 0.0}"
20 - 29,Private,HS-grad,Never-married,Sales,Not-in-family,Female,35,United-States,<=50K,"{'<=50K': 4.169193524701747e-06, '>50K': 0.0}"
40 - 49,Private,Some-college,Widowed,Adm-clerical,Not-in-family,Female,60,United-States,<=50K,"{'<=50K': 5.323166732431695e-12, '>50K': 0.0}"
40 - 49,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,Male,45,United-States,<=50K,"{'<=50K': 7.777353992189164e-06, '>50K': 0.0}"
